# ОБЪЕДИНЕНИЕ ТАБЛИЦ – JOIN

Описание данных:
    
`installs` — содержит данные об установках приложения по дням.

· DeviceID — идентификатор устройства, на которое было установлено приложение;  
· InstallationDate — дата установки приложения;  
· InstallCost — цена установки приложения в рублях;  
· Platform — платформа, на которой было установлено приложение (iOS/ Android);  
· Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).  

`events` — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.  

· DeviceID — идентификатор устройства, на котором используется приложение;  
· AppPlatform — платформа, на которой используется приложение (iOS/ Android);  
· EventDate — дата, за которую собрана статистика;  
· events — количество просмотров всех товаров за этот день у этого DeviceID.  
  
`checks` — содержит данные о покупках пользователей в приложении по дням  

· UserID — идентификатор пользователя;  
· Rub — суммарный чек пользователя на дату;  
· BuyDate — дата, за которую собрана статистика.  

Особенность приложения заключается в том, что для просмотра товаров не нужна авторизация. До момента авторизации про пользователя известен только его DeviceID — идентификатор устройства. При этом для совершения покупки логин обязателен. На моменте авторизации пользователю присваивается UserID, и тогда мы уже знаем два его идентификатора: DeviceID (устройство) и UserID (логин). Так как на этапах установки приложения и просмотра каталога пользователь еще может быть не авторизован, там мы сохраняем только DeviceID. Но так как покупки нельзя совершить без авторизации, то покупки сохраняются только с UserID. Для того чтобы просмотры и установки можно было объединить с покупками, нам нужна таблица соответствия DeviceID к UserID, то есть таблица `devices`:  

· DeviceID — идентификатор устройства;  
· UserID — идентификатор пользователя.

### 1

У пользователя может быть два идентификатора – UserID и DeviceID. В таблице checks есть только UserID, в остальных – только DeviceID. Во вспомогательной таблице devices есть и UserID, и DeviceID. 

Давайте с помощью JOIN дополним таблицу events (left) данными о UserID пользователей из таблицы devices (right). Для некоторых DeviceID не будет пары UserID из таблицы devices – подумайте, какой вид JOIN подойдет, чтобы не потерять те строки, где DeviceID есть в events, но нет в devices.

Укажите UserID из первой строки результирующей таблицы, используя сортировку по убыванию по полю DeviceID.

In [ ]:
SELECT e.AppPlatform AS AppPlatform,
       e.events AS events,
       e.EventDate AS EventDate,
       e.DeviceID AS DeviceID,
       d.UserID AS UserID
  FROM events AS e 
       LEFT JOIN devices AS d
       on e.DeviceID = d.DeviceID
 ORDER BY DeviceID DESC
 LIMIT 1

### 2

Давайте проверим, пользователи пришедшие из какого источника совершили наибольшее число покупок. В качестве ответа выберите название Source, юзеры которого совершили больше всего покупок.

Hint: Для этого используйте UserID, DeviceID и Source из соответствующих таблиц. Считать уникальные значения здесь не нужно.

In [ ]:
SELECT i.Source AS Source,
       COUNT(c.UserID) AS cnt
  FROM checks AS c
  JOIN devices AS d
    ON c.UserID = d.UserID
  JOIN installs AS i
    ON d.DeviceID = i.DeviceID
 GROUP BY i.Source
 ORDER BY cnt DESC
 LIMIT 1

### 3

Теперь выясним, сколько всего уникальных юзеров что-то купили в нашем приложении.

Объедините нужные таблицы, посчитайте число уникальных UserID для каждого источника (Source), и в качестве ответа укажите число пользователей, пришедших из Source_7.

Hint: checks – покупки, devices – соответствие, installs – информация об источнике.

In [ ]:
SELECT i.Source AS Source,
       COUNT(DISTINCT c.UserID) AS uniq_users
  FROM checks AS c
  JOIN devices AS d
    ON c.UserID = d.UserID
  JOIN installs AS i
    ON d.DeviceID = i.DeviceID
 WHERE Source = 'Source_7'
 GROUP BY i.Source

### 4

Самое время посмотреть на общую выручку, а также минимальный, максимальный и средний чек. Рассчитайте нужные показатели.

SELECT i.Source AS Source,
       SUM(Rub) as Revenue,
       MIN(Rub) as MinRub,
       MAX(Rub) AS MaxRub,
       AVG(Rub) as AvgRub
  FROM checks as c
  JOIN devices as d
    ON c.UserID = d.UserID
  JOIN installs as i
    ON d.DeviceID = i.DeviceID
 GROUP BY i.Source

### 5

Выведите идентификаторы устройств пользователей, которые совершили как минимум одну покупку за последний месяц (октябрь 2019). Используйте сортировку по возрастанию DeviceID и укажите минимальное значение.

Hint: для извлечения месяца из даты можно использовать toMonth() или  toStartOfMonth(), предварительно приведя BuyDate к типу date.

In [ ]:
SELECT *
  FROM checks AS c
  JOIN devices AS d
    ON c.UserID = d.UserID
 WHERE toStartOfMonth(CAST(BuyDate AS date)) = '2019-10-01'
 ORDER BY DeviceID
 LIMIT 10

### 6

Проверим, сколько товаров (events) в среднем просматривают пользователи с разных платформ (Platform), и пришедшие из разных источников  (Source). Для этого объедините таблицы events и installs, и посчитайте, сколько просмотров в среднем приходится на каждую пару платформа-канал привлечения.

Отсортируйте полученную табличку по убыванию среднего числа просмотров. 

In [ ]:
SELECT e.AppPlatform AS AppPlatform,
       i.Source AS Source,
       AVG(e.events) AS avg_events
  FROM events AS e
  JOIN installs AS i
    ON e.DeviceID = i.DeviceID

 GROUP BY e.AppPlatform,
         i.Source 
 ORDER BY avg_events DESC
 LIMIT 10

### 7

Давайте посчитаем число уникальных DeviceID в инсталлах, для которых присутствуют просмотры в таблице events с разбивкой по платформам (поле Platform). Для этого можно отобрать все строки только из таблицы installs, для которых нашлось соответствие в таблице events. 

In [ ]:
SELECT Platform,
       Count(distinct DeviceID) AS devices_nubmer
  FROM installs AS i 
       LEFT SEMI JOIN events AS e 
       ON i.DeviceID = e.DeviceID 
 GROUP BY i.Platform
 LIMIT 10

### 8

Давайте теперь посчитаем конверсию из инсталла в просмотр с разбивкой по платформе инсталла – в данном случае это доля DeviceID, для которых есть просмотры, от всех DeviceID в инсталлах. 

Для этого нужно объединить таблицы installs и events так, чтобы получить все DeviceID инсталлов и соответствующие им DeviceID из events, посчитать число уникальных DeviceID инсталлов (1) и соответствующих DeviceID из events (2) и вычислить долю (2) от (1). В качестве ответа укажите значение конверсии из инсталла в просмотр на платформе ios. 

In [ ]:
SELECT Platform, 
       COUNT(DISTINCT ev.DeviceID) / COUNT(DISTINCT i.DeviceID) AS Conversion
  FROM installs AS i 
       LEFT JOIN events AS e
       ON i.DeviceID = e.DeviceID 
 GROUP BY Platform

### 9

Представим, что в логирование DeviceID в событиях закралась ошибка - часть ID была записана в базу некорректно. Это привело к тому, что в таблице с событиями появились DeviceID, для которых нет инсталлов. Нам надо отобрать примеры DeviceID из таблицы event, которых нет в таблице installs, чтобы отправить их команде разработчиков на исправление. 

Выведите 10 уникальных DeviceID, которые присутствуют в таблице events, но отсутствуют в installs, отсортировав их в порядке убывания.

In [ ]:
SELECT DISTINCT(DeviceID)
  FROM events AS e
       LEFT ANTI JOIN installs AS i
       ON i.DeviceID = e.DeviceID
 ORDER BY DeviceID DESC
 LIMIT 10